In [1]:
# 1.LIBRERIAS
import numpy as np
import pandas as pd
import pickle
from janitor import clean_names

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import HistGradientBoostingRegressor

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline


# 1.CARGA DATOS
ruta_proyecto = "C:/Users/Francisco/OneDrive/DATA_SCIENCE_MASTERY/03_MACHINE_LEARNING/08_CASOS/07_PERDIDA_ESPERADA"
nombre_fichero_datos = "validacion.csv"
ruta_completa = ruta_proyecto + "/02_Datos/02_Validacion/" + nombre_fichero_datos
df = pd.read_csv(ruta_completa, index_col=0)


# 3.VARIABLES Y REGISTROS FINALES
variables_finales = [
    "ingreso_mes",
    "gasto_mes",
    "activos",
    "pasivos",
    "antiguedad_empleo",
    "vivienda",
    "num_hipotecas",
    "porc_tarjetas_75p",
    "score_credito",
    "proposito_credito",
    "principal",
    "numero_cuotas",
    "interes",
    "cuota",
    "impago",
    "amortizado",
    "recuperada",
]

df.drop_duplicates(inplace=True)

df = df[variables_finales]

df = df.fillna(0)

# 4.FUNCIONES DE SOPORTE


def calidad_datos(temp):
    for column in temp.select_dtypes("number").columns:
        temp[column] = temp[column].fillna(0)
    return temp


def creacion_variables(df):
    temp = df.copy()
    return temp


# 4.CALIDAD Y CREACION DE VARIABLES
x_pd = creacion_variables(calidad_datos(df))
x_ead = creacion_variables(calidad_datos(df))
x_lgd = creacion_variables(calidad_datos(df))


# 5.CARGA PIPES DE EJECUCION
ruta_pipe_ejecucion_pd = ruta_proyecto + "/04_Modelos/pipe_ejecucion_pd.pickle"
ruta_pipe_ejecucion_ead = ruta_proyecto + "/04_Modelos/pipe_ejecucion_ead.pickle"
ruta_pipe_ejecucion_lgd = ruta_proyecto + "/04_Modelos/pipe_ejecucion_lgd.pickle"

with open(ruta_pipe_ejecucion_pd, mode="rb") as file:
    pipe_ejecucion_pd = pickle.load(file)

with open(ruta_pipe_ejecucion_ead, mode="rb") as file:
    pipe_ejecucion_ead = pickle.load(file)

with open(ruta_pipe_ejecucion_lgd, mode="rb") as file:
    pipe_ejecucion_lgd = pickle.load(file)


# 6.EJECUCION
scoring_pd = pipe_ejecucion_pd.predict_proba(x_pd)[:, 1]
ead = pipe_ejecucion_ead.predict(x_ead)
lgd = pipe_ejecucion_lgd.predict(x_lgd)


# 7.RESULTADO
principal = x_pd.principal
EL = pd.DataFrame({"principal": principal, "pd": scoring_pd, "ead": ead, "lgd": lgd})
EL["perdida_esperada"] = round(EL.pd * EL.principal * EL.ead * EL.lgd, 2)

In [2]:
EL

,principal,pd,ead,lgd,perdida_esperada
id_cliente,,,,,
1325609,13958.144919,0.043166,0.005598,0.009075,0.03
65615352,23389.524921,0.005606,0.009740,0.009288,0.01
93981272,14380.153807,0.018237,0.011677,0.026236,0.08
51987698,23888.062855,0.690325,0.565080,0.962141,8965.69
138332093,13625.662133,0.017717,0.006466,0.009956,0.02
...,...,...,...,...,...
76936043,11664.599266,0.035001,0.174619,0.180788,12.89
62176153,16110.092377,0.045782,0.025668,0.025821,0.49
96930390,4118.768743,0.030901,0.010079,0.016102,0.02


In [3]:
df

,ingreso_mes,gasto_mes,activos,pasivos,antiguedad_empleo,vivienda,num_hipotecas,porc_tarjetas_75p,score_credito,proposito_credito,principal,numero_cuotas,interes,cuota,impago,amortizado,recuperada
id_cliente,,,,,,,,,,,,,,,,,
1325609,1958.333333,1240.436905,142047.0,7369.60,< 1 año,ARRENDADA,0.0,50.0,873,otros,13958.144919,36,15.29,485.85,0,13958.144919,0.000000
65615352,2412.500000,1209.528080,108117.0,8725.53,5 años,PROPIA NO HIPOTECADA,0.0,0.0,935,pago_tarjeta_credito,23389.524921,36,16.31,825.89,0,23389.524921,0.000000
93981272,2625.000000,1737.478532,112311.0,21142.80,10+ años,PROPIA NO HIPOTECADA,0.0,50.0,748,consolidar_deudas,14380.153807,30,15.53,581.45,0,14380.153807,0.000000
51987698,2020.833333,1201.758158,25310.0,10417.80,4 años,HIPOTECADA,4.0,80.0,772,consolidar_deudas,23888.062855,54,16.17,625.51,1,12965.377265,1453.527693
138332093,1458.333333,617.377957,185540.0,3409.00,< 1 año,ARRENDADA,0.0,33.3,865,pago_tarjeta_credito,13625.662133,30,15.14,548.38,0,13625.662133,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76936043,2416.666667,1216.797458,116702.0,11774.00,DESCONOCIDO,HIPOTECADA,3.0,20.0,969,consolidar_deudas,11664.599266,18,14.52,725.06,0,11664.599266,0.000000
62176153,4500.000000,2014.273313,38359.0,24861.60,10+ años,ARRENDADA,1.0,80.0,844,consolidar_deudas,16110.092377,12,14.61,1451.11,0,16110.092377,0.000000
96930390,2500.000000,1864.489468,104689.0,9420.00,2 años,ARRENDADA,0.0,16.7,913,consolidar_deudas,4118.768743,12,15.88,373.47,0,4118.768743,0.000000
